In [18]:
import getpass
user_id = getpass.getuser()

import sys
sys.path.append(f"/home/{user_id}/OTTEHR/")
sys.path.append(f"/home/{user_id}/Statistical-Fairness-Test")

from collections import Counter
from common import *
from googletrans import Translator
import matplotlib.pyplot as plt
from mimic_common import *
from sklearn.decomposition import PCA

base_path = f"/home/{user_id}/OTTEHR/mimiciv_exp"
output_path = f"/home/{user_id}/OTTEHR/outputs/mimiciv"

The goal is to run Taskesen's statistical test to study the relationship between explanatory variable ICD code and label Insolin prescription based on protected attribute race (black and white)

Step 1: Select source and target samples

In [19]:
df = pd.read_csv(os.path.join(base_path, "mimiciv/final_admission.csv"), converters={'diagnosis_icd_code': literal_eval, 'procedure_icd_code': literal_eval, 'drug': literal_eval})
display(df)

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,...,edouttime,hospital_expire_flag,gender,anchor_age,anchor_year,anchor_year_group,dod,diagnosis_icd_code,procedure_icd_code,drug
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,...,2180-05-06 23:30:00,0,F,52,2180,2014 - 2016,2180-09-09,"[5723, 78959, 5715, 07070, 496, 29680, 30981, ...",[5491],"[Acetaminophen, Sodium Chloride 0.9% Flush, F..."
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,EW EMER.,P09Q6Y,EMERGENCY ROOM,HOME,Medicaid,...,2180-06-26 21:31:00,0,F,52,2180,2014 - 2016,2180-09-09,"[07071, 78959, 2875, 2761, 496, 5715, V08, 3051]",[5491],"[Rifaximin, Lactulose, Calcium Carbonate, Furo..."
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaN,EW EMER.,P60CC5,EMERGENCY ROOM,HOSPICE,Medicaid,...,2180-08-06 01:44:00,0,F,52,2180,2014 - 2016,2180-09-09,"[07054, 78959, V462, 5715, 2767, 2761, 496, V0...",[5491],"[Albumin 25% (12.5g / 50mL), Calcium Carbonate..."
3,10000117,27988844,2183-09-18 18:10:00,2183-09-21 16:30:00,NaN,OBSERVATION ADMIT,P75S70,WALK-IN/SELF REFERRAL,HOME HEALTH CARE,Other,...,2183-09-18 20:20:00,0,F,48,2174,2008 - 2010,NaN,"[S72012A, W010XXA, Y93K1, Y92480, K219, E7800,...",[0QS734Z],"[Senna, Bisacodyl, Cepacol (Sore Throat Lozeng..."
4,10000560,28979390,2189-10-15 10:30:00,2189-10-17 15:00:00,NaN,SURGICAL SAME DAY ADMISSION,P08ZLF,PHYSICIAN REFERRAL,HOME,Other,...,NaN,0,F,53,2189,2008 - 2010,NaN,"[1890, V1582, V1201]",[5551],"[Scopolamine Patch, Ketorolac, Lorazepam, Pota..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220848,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,NaN,EW EMER.,P75BG6,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,...,2149-01-08 18:12:00,0,F,46,2147,2017 - 2019,NaN,"[T8141XA, E1110, K632, D682, L02211, T83728A, ...","[0J980ZZ, 0WPF0JZ, 05HY33Z]","[Insulin, Potassium Chloride, Insulin, Insulin..."
220849,19999828,29734428,2147-07-18 16:23:00,2147-08-04 18:10:00,NaN,EW EMER.,P16C7J,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,...,2147-07-18 17:34:00,0,F,46,2147,2017 - 2019,NaN,"[T8131XA, T8183XA, K632, D6851, N390, Y838, Y9...","[0HR7X74, 0HBJXZZ, 0HBHXZZ, 02HV33Z, 3E0436Z]","[Insulin, Potassium Chloride (Powder), Acetami..."
220850,19999840,21033226,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,EW EMER.,P58A9J,EMERGENCY ROOM,DIED,Other,...,2164-09-10 14:46:00,1,M,58,2164,2008 - 2010,2164-09-17,"[3453, 51881, 5070, 5180, 42741, 43821, 43811,...","[9604, 9672, 966, 0331, 0331]","[Potassium Chloride, Bisacodyl, Phenytoin, 0.9..."
220851,19999840,26071774,2164-07-25 00:27:00,2164-07-28 12:15:00,NaN,EW EMER.,P506DE,EMERGENCY ROOM,HOME,Other,...,2164-07-25 01:20:00,0,M,58,2164,2008 - 2010,2164-09-17,"[43491, 43820, 34590, 43811, 4019, 2724, 3051]","[8891, 8841]","[Metoprolol Tartrate, Sodium Chloride 0.9% Fl..."


In [20]:
def select_drug_samples(df, group_name, drug, source, target, source_count = None, target_count = None):
    """ 
    Select rows in the dataframe df for source and target samples, return the results in \
        a format that is compatible to Taskesen et al's statistical test 
    
    :param df Dataframe: the dataframe to select samples from.
    :param str drug: drug name, used for preparing the labels array
    :param str source: source group name, e.g., White
    :param str target: target group name, e.g., Black
    :param int source_count: if specified, the number of samples for the source. \
        Otherwise, delete all rows that do not belong to target or source
    :param int target_count: if specified, the number of samples for the target. \
        Otherwise, delete all rows that do not belong to target or source
    """
    df_copy = copy.deepcopy(df)
    
    source_indices = []
    target_indices = []
    other_indices = []
    for index, row in df_copy.iterrows():
        if row[group_name] == source:
            source_indices.append(index)
        elif row[group_name] == target:
            target_indices.append(index)
        else:
            other_indices.append(index)

    
    # indices to delete from the dataframe
    if source_count is not None:
        delete_source_indices = random.sample(source_indices, len(source_indices)-source_count)
        delete_target_indices = random.sample(target_indices, len(target_indices)-target_count)

        other_indices.extend(delete_source_indices)
        other_indices.extend(delete_target_indices)

    df_copy = df_copy.drop(other_indices, axis=0, inplace=False)

    # prepare labels array, indicator of label attributes and \
    # sensitive_labels array: indicator of protected attributes
    labels = []
    sensitive_labels = []
    for _, row in df_copy.iterrows():
        if drug in row['drug']:
            labels.append(1)
        else:
            labels.append(0)

        if row[group_name] == source:
            sensitive_labels.append(1)
        elif row[group_name] == target:
            sensitive_labels.append(0)

    return df_copy, np.array(labels), np.array(sensitive_labels)

In [21]:
source_num = 100
target_num = 100
selected_df, labels, sensitive_labels = select_drug_samples(df, 'race', 'Insulin', 'WHITE', 'ASIAN', source_num, target_num)

In [22]:
display(selected_df)

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,...,edouttime,hospital_expire_flag,gender,anchor_age,anchor_year,anchor_year_group,dod,diagnosis_icd_code,procedure_icd_code,drug
682,10031358,27421511,2156-09-06 18:22:00,2156-09-17 17:54:00,NaN,EW EMER.,P77YFC,EMERGENCY ROOM,HOME HEALTH CARE,Other,...,2156-09-06 20:18:00,0,M,58,2152,2008 - 2010,NaN,"[25080, 73007, 7854, 2761, 70715, 5849, 25060,...","[8411, 8411, 8604, 3897]","[Atorvastatin, Sodium Chloride 0.9% Flush, He..."
862,10039302,29564464,2124-01-29 21:46:00,2124-02-06 17:51:00,NaN,EW EMER.,P90LOA,EMERGENCY ROOM,REHAB,Medicare,...,2124-01-30 11:59:00,0,F,72,2118,2008 - 2010,2124-08-05,"[99674, 44024, 3572, 70714, 6827, E8782, E8497...",[8415],"[Sodium Chloride 0.9%, Sodium Chloride 0.9% F..."
3770,10175944,28061875,2155-08-20 22:16:00,2155-08-29 19:00:00,NaN,URGENT,P08BV8,TRANSFER FROM HOSPITAL,HOSPICE,Other,...,2155-08-20 23:24:00,0,F,47,2148,2008 - 2010,2155-08-30,"[A419, K767, J9690, G9340, N179, D689, K567, E...","[0D568ZZ, 0W9G3ZZ, 0W9G3ZZ]","[Midazolam, Lactulose, Rifaximin, Bag, Magnesi..."
4383,10207476,21122306,2171-03-05 08:32:00,2171-03-06 17:32:00,NaN,EU OBSERVATION,P54631,EMERGENCY ROOM,NaN,Other,...,2171-03-05 11:23:00,0,F,63,2169,2008 - 2010,NaN,"[78791, 79431, 41400, 20501, V4281, 49390, 530...","[8952, 8801, 8838, 8938, 3899]","[Metoprolol Tartrate, Albuterol Inhaler, Atorv..."
4926,10232463,24442495,2137-03-19 00:00:00,2137-03-21 18:35:00,NaN,SURGICAL SAME DAY ADMISSION,P27180,PHYSICIAN REFERRAL,HOME,Medicaid,...,NaN,0,F,62,2137,2014 - 2016,NaN,"[C530, M069, R339, Z8049]","[0UT94ZZ, 0UTC4ZZ, 0UT24ZZ, 0UT74ZZ, 0TJB8ZZ]","[OxycoDONE (Immediate Release) , Docusate Sodi..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217541,19851462,20330525,2129-03-24 14:45:00,2129-04-08 17:30:00,NaN,EW EMER.,P53XKM,EMERGENCY ROOM,HOME HEALTH CARE,Other,...,2129-03-24 16:14:00,0,F,43,2128,2008 - 2010,2129-05-22,"[4238, 486, 5128, 1623, 1983, 2762, 5180, 5118...","[3712, 3712, 370, 3404, 9671, 3891, 3897, 3897...","[5% Dextrose, Phenylephrine, Heparin, Prochlor..."
217587,19854456,24548604,2177-07-30 00:23:00,2177-08-04 14:21:00,NaN,EW EMER.,P257ZL,EMERGENCY ROOM,SKILLED NURSING FACILITY,Medicare,...,2177-07-30 01:34:00,0,M,87,2177,2008 - 2010,2178-05-30,"[56039, 1977, 1538, 1985, 27651, 185, V441, V1...",[3893],"[Multivitamins, Prochlorperazine, 0.9% Sodium ..."
217786,19862113,27598508,2181-01-12 22:13:00,2181-01-15 14:00:00,NaN,EW EMER.,P387UT,PHYSICIAN REFERRAL,HOME,Other,...,2181-01-12 23:21:00,0,F,62,2181,2017 - 2019,NaN,"[S0912XA, S12301A, S37012A, S06899A, S2242XA, ...","[0KQ10ZZ, 08QPXZZ]","[Magnesium Oxide, Acetaminophen IV, HYDROmorph..."
217866,19865167,24112281,2152-03-20 08:45:00,2152-03-22 15:25:00,NaN,SURGICAL SAME DAY ADMISSION,P13WAF,PHYSICIAN REFERRAL,HOME,Other,...,NaN,0,F,57,2152,2011 - 2013,NaN,"[73341, 71811]","[8181, 8388]","[OxycoDONE (Immediate Release) , Omeprazole, T..."


In [23]:
Counter(labels)

Counter({0: 149, 1: 51})

In [24]:
Counter(sensitive_labels)

Counter({1: 100, 0: 100})

In [25]:
def gen_feature(df, feature_code_name):
    """ 
    Generate source features and target features from dataframe df
    :param str feature_code_name: name of the input. For the experiments on ICD codes vs. duration, the input_name is 'ICD codes'

    """
    unique_code_dict, num_codes = find_unique_code(df, ICD_name = feature_code_name)

    # Prepare source
    features = np.empty(shape=[df.shape[0], num_codes])
    feature_index = 0
    for _, row in df.iterrows():
        code_ind = np.zeros(num_codes)
        for code in row[feature_code_name]:
            code_ind[unique_code_dict[code]] += 1
        features[feature_index] = code_ind
        feature_index += 1

    return features

In [26]:
features = gen_feature(selected_df, 'diagnosis_icd_code')

In [27]:
pca = PCA(n_components=6)
reps = pca.fit_transform(features)


In [28]:
print(features.shape, reps.shape)

(200, 1055) (200, 6)


Step 2: Run Taskesen's statistical test

In [29]:
from load_data import *
from matplotlib import colors as mcolors
from random import seed
from functions import *
import time
from LogisticRegression import LogisticRegression
import datetime
from scipy.stats import gamma

In [30]:
clf = LogisticRegression(fit_intercept=False)
clf.fit(X=reps, y=labels)
beta = clf.coef_
print(beta)

[ 1.38178984  1.07910812 -0.32259678  0.40028306  1.31032098  0.22191151]


In [31]:
data_tuple = [reps, np.array(sensitive_labels), np.array(labels)]
range_gamma = [-5, 5]
range_of_k = np.linspace(0, 1/8, 100)
tol = 1e-10

In [32]:
test_marginals = np.reshape(get_marginals(sensitives=sensitive_labels, target=labels), [2, 2])
c = limiting_dist_EQOPP(X=reps, a=sensitive_labels, y=labels, beta=beta, marginals=test_marginals)
theta = 2 * c
sig_value = gamma.ppf(.95, a=1/2, scale=theta)
print(f"The rejection significance value is {sig_value}")

The rejection significance value is 1.2997287088248626


In [33]:
dist, optimum_ks, gamma_opt = calculate_distance_prob_eqopp_with_opt_params(data_tuple=data_tuple,
                                          function_of_gamma=f_gamma,
                                          range_gamma=range_gamma,
                                          k_opt_fcn=k_opt,
                                          range_of_k=range_of_k,
                                          beta=beta, tol=tol)

gamma is -1.1803398874989481, lambd is 6.896551724137931, LA.norm(beta, ord=2) is 2.2592694632795616, k is 0.0, beta.T @ x_hat is 2.9358463981907303, the term within exp is: -2.9358463981907303\gamma is -1.1803398874989481, lambd is -9.09090909090909, LA.norm(beta, ord=2) is 2.2592694632795616, k is 0.0, beta.T @ x_hat is 0.7857992912883225, the term within exp is: -0.7857992912883225\gamma is -1.1803398874989481, lambd is -9.09090909090909, LA.norm(beta, ord=2) is 2.2592694632795616, k is 0.0, beta.T @ x_hat is 0.2212567607846687, the term within exp is: -0.2212567607846687\gamma is -1.1803398874989481, lambd is 6.896551724137931, LA.norm(beta, ord=2) is 2.2592694632795616, k is 0.0, beta.T @ x_hat is 4.492980726199508, the term within exp is: -4.492980726199508\gamma is -1.1803398874989481, lambd is 6.896551724137931, LA.norm(beta, ord=2) is 2.2592694632795616, k is 0.0, beta.T @ x_hat is 2.5286578676954923, the term within exp is: -2.5286578676954923\gamma is -1.1803398874989481, la

In [ ]:
print(dist)

0.005469110936927488
